In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
def login(login, psswd, driver):
    driver.get("https://reservation.42network.org/signin")
    login_field = driver.find_element_by_id("user_login")
    psswd_field = driver.find_element_by_id("user_password")
    login_field.clear()
    login_field.send_keys(login)
    psswd_field.clear()
    psswd_field.send_keys(psswd)
    psswd_field.send_keys(Keys.RETURN)

In [3]:
def logout(driver):
    driver.find_elemt_by_xpath("//*[@id='navbar-main']/div[2]/a[2]").click()
    driver.find_element_by_xpath("//span[text()='logout']").click()    

In [96]:
def get_week_days(driver):
#     return driver.find_elements_by_xpath("//div[@class='fc-content-skeleton']/table/tbody/tr/td")
    return driver.find_elements_by_xpath("//div[@class='fc-content-skeleton']//div[@class='fc-event-container']")
#     return driver.find_elements_by_xpath("//div[@class='fc-content-skeleton']")

In [5]:
driver = webdriver.Chrome("./chromedriver")
login("jbarment", "69@TheEelHouse!", driver)

In [74]:
weekdays = get_week_days(driver)
mday = weekdays[3]

In [91]:
for el in weekdays:
    print(el.get_attribute("class"))

fc-event-container
fc-event-container
fc-event-container
fc-event-container
fc-event-container
fc-event-container
fc-event-container


In [87]:
mday = weekdays[3]

In [102]:
# len(mday.find_elements_by_xpath("//a[@class=*]"))
def get_elements_in_day(day_element):
    return day_element.find_elements_by_xpath(".//a")


In [103]:
# a = driver.find_element_by_xpath("//*[@id='app']/div[2]/section/div[2]/div[2]/div/table/tbody/tr/td/div/div/div[3]/table/tbody/tr/td[5]/div/div[2]/a[31]")
# a.get_attribute("class")
len(mday.find_elements_by_xpath(".//a[@class='fc-time-grid-event fc-event fc-start fc-end']"))
els = get_elements_in_day(get_week_days(driver)[5])
len(els)

33

In [90]:
els[4].click()